# 텐서플로 세션의 구조

텐서플로는 머신러닝 과정을 그래프로 나타내고 있고, 이를 실행하는 과정을 분리해두었습니다. 그래프를 구축하는 단계에서는 데이터 자체의 의미보다 데이터의 형태(모양)에 더 신경을 쓰게 되고, 실행과정에서는 이 형태에 맞는 데이터들을 그래프에 흘려보내는 방식입니다.

In [1]:
import tensorflow as tf

/Users/skim0119/Desktop/Project/ML_Library_Notes/venv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## 그래프

텐서플로 라이브러리를 사용하기 위해 가장 먼저 이해해야 하는 개념은 그래프와 세션입니다. C/C++같은 경우 개발 과정은 결국 코딩, 컴파일, 실행 과정으로 나뉠 수 있습니다. 코딩 단계에서 알고리즘과 프로그램의 구조를 결정하고, 그 코드를 컴파일 하고 나면 그 이후는 컴파일된 프로그램을 실행하는 구조입니다. 텐서플로는 근본적으로 이러한 방식의 개발과정을 따라간다고 볼 수 있습니다. 언어 자체는 파이썬을 사용하지만 개발 과정은 하나의 그래프를 설계한 뒤 그 그래프를 실행해나가는 수순입니다.

이미지분류에 자주 사용되는 네트워크를 살펴보도록 하겠습니다.

![image conv networks](https://cdn-images-1.medium.com/max/1200/1*2ns4ota94je5gSVjrpFq3A.png)

위의 그래프는 이미지라는 데이터를 주어졌을때부터 분류라는 결과가 나올때까지의 데이터 흐름을 보여주고있습니다. 여기서 데이터는 다차원 매트릭스, 혹은 텐서입니다. 각 블록은 이 데이터가 어떠한 계산을 행하는지를 보여주고, 입력과 결과의 사이즈가 어떻게 바뀌는지를 보여주고있습니다.

그렇다면 왜 텐서플로는 이러한 방식을 택했는지를 생각해보겠습니다.

우선 머신러닝의 특성상 컴퓨터는 수많은 똑같은 계산과정을 실행해야합니다. 그래서 모든 계산과정을 하나의 블랙박스 함수로 만든다음, 이 함수에 방대한 양의 데이터를 실행하는 형식으로 진행됩니다. 라이브러리 내에서는 이 거대한 함수를 어떻게 효율적으로 처리할 것인지에 대해 많은 최적화 방법들이 적용되어 있어 그래픽카드 혹은 TPU를 사용했을 때 일반적인 계산보다 훨씬 빠르게 처리가 가능합니다.

계산과정을 그래프로 나타낼 경우, 데이터의 흐름이 명확해집니다. 각 노드를 계산식이나 함수라 생각하고, 그 외에 상수, 변수, 행렬, 텐서(고차원행렬)이 노드들을 정해진 수순대로 진행해나간다고 생각할 수 있습니다. 즉 텐서플로로 구축된 모델은 하나의 시작점에서 시작한 방향성 그래프라고 보면 됩니다. 이는 각 데이터 자체의 의미보다 데이터의 모양과 흐름을 나타내는데에 더 집중할 수 있게 도와줍니다.

간단한 예제를 만들어보겠습니다.

In [15]:
tf.reset_default_graph()

a = tf.constant(5)
b = tf.constant(6)
c = tf.constant(7)
d = (a + b) * c

for i in tf.get_default_graph().get_operations():
    print(i.name + '\n')

Const

Const_1

Const_2

add

mul



위의 그래프는 상수를 나타내는 const 3개와 덧샘(add)한개, 곱샘(mul)한개로 이루어져 있습니다. 한가지 흥미로운점은 d는 곱샘(mul)이라는 점입니다.

In [17]:
d

<tf.Tensor 'mul:0' shape=() dtype=int32>

텐서플로 라이브러리 안에서 텐서는 상당히 범용적인 의미로 사용된다는 것을 알 수 있습니다. d는 곱샘을 나타내는 노드라 할 수 있지만, 결과물의 모양과 데이터형을 가진 하나의 텐서로 보고있습니다. 즉 텐서플로는 변수나 상수만 텐서로 나타내는게 아니라 모든 계산과정 또한 텐서로 나타낸다고 볼 수 있습니다.

## 세션

세션은 그래프나 그래프의 일부분을 '실행'하는 공간입니다. 텐서플로가 이렇게 계산형태와 계산과정을 분리한 이유는 머신러닝의 특성과 밀접하게 관련이 있습니다. 머신러닝은 기본적으로 '모델링'과 '트레이닝,' 그리고 '예측'의 과정으로 이루어져있습니다. 그렇다보니 '계산의 결과'보다 '계산의 형태'를 저장하고 수정하는게 더 효과적일 수 있습니다.